# L1: stochastic block model and community detection

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import time

In [ ]:
import sys
sys.path.append('../../../src/')
import tools as tl
import plot as viz
import pysbm

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

colormap = plt.cm.tab20
colors = {i: colormap(i) for i in range(20)}

In [ ]:
from probinet.input.loader import build_adjacency_from_file
from probinet.input.stats import print_graph_stats
from probinet.models.mtcov import MTCOV
from probinet.visualization.plot import plot_hard_membership, plot_soft_membership


In [ ]:
outdir = '../figures/'
lecture_id = 1

In [ ]:
seed = 10
prng = np.random.RandomState(seed)

# 1. Trade network
Let's consider a trade network. You can find various examples online, in the website of FAO or of the World Trade Organization (WTO).   
In this example we take a network downloaded from WTO, but feel free to use your favorite network.  
If you want to use the same dataset in this example: below you find the instructions to download the data.

## 1.1 Download the [raw data](https://ttd.wto.org/en/download/six-digit?years[0]=2024&indicator=exports&products[0]=271600) locally in some folder.  
    Filter by:

       a. Year = 2024
       b. Items = Electrical energy (271600)


## 1.2 Preprocess the data
To clean and build a simple edge list 

In [ ]:
indir = '../../../data/input/wto/'
filename = 'adb_exports_08_04_2025_13_57_41.csv'
infile = f"{indir}{filename}"

df0 = pd.read_csv(infile)
df0.head(n=3)

In [ ]:
df0.product_code.unique()

### 1.2.1 Filter nodes and discretize weights

In [ ]:
cond1 = (df0.reporter_name != 'World') & (df0.partner_name != 'World')
cond2 = df0.product_code == df0.product_code.unique()[0] # keep only one type of good
mask = cond1 & cond2
df0 = df0[mask]
len(df0)

#### Top exporters

In [ ]:
df0.groupby(by='reporter_name')['value'].agg('sum').sort_values(ascending=False).iloc[:10]

#### Top importers

In [ ]:
df0.groupby(by='partner_name')['value'].agg('sum').sort_values(ascending=False).iloc[:10]

#### Top links

In [ ]:
df0.sort_values(by='value',ascending=False).iloc[:10][['reporter_name','partner_name','value']]

In [ ]:
senders = df0['reporter_name'].unique()
receivers = df0['partner_name'].unique()
nodes = set(senders).union(set(receivers))
N = len(nodes)
N

We make all weights discrete, as we want to focus on the _existence_ of relationships (not on their strength)

In [ ]:
df0['weight']  = 1

### 1.2.2 Combine repeated edges and make undirected
Some pairs of countries may appear more than once, we need to combine them into 1 edge.  
We are also discarding edge directions.

In [ ]:
source = 'reporter_name'
target = 'partner_name'
weight= 'weight'

In [ ]:
df_reverse = pd.DataFrame({source: df0[target], target: df0[source], weight: df0[weight]})
df = pd.concat([df0, df_reverse])

df = df.drop_duplicates()
df = df.groupby(by=[source, target]).aggregate({weight: 'sum'}).reset_index()
df['weight']  = 1

In [ ]:
print(f"E = {len(df)}")
df.head()

### 1.2.3 Keep largest connected component

In [ ]:
graph = nx.from_pandas_edgelist(df,source=source,target=target,
                               edge_attr=weight,create_using=nx.Graph)

Gcc = sorted(nx.connected_components(graph), key=len, reverse=True)
graph = graph.subgraph(Gcc[0])

nodes = list(graph.nodes())
nodes.sort()

cond1 = df[source].isin(nodes)
cond2 = df[target].isin(nodes)
mask = cond1 & cond2

df = df[mask]
len(nodes)

In [ ]:
graph = nx.from_pandas_edgelist(df,source='reporter_name',target='partner_name',
                               edge_attr='weight',create_using=nx.Graph)
Gcc = sorted(nx.connected_components(graph), key=len, reverse=True)
assert len(Gcc[0]) == len(nodes)

### 1.2.4 Save into a file for future use

In [ ]:
outdir = '../../../data/output/wto/'
filename = 'wto_aob.csv'

tl.save_df_to_file(df,outdir=outdir,filename=filename)

In [ ]:
!head ../../../data/outdir/wto/wto_aob.csv

### 1.2.5 Extract adjacency matrix Y
We can arrange the dataset into an adjacency matrix, the main input data

We can use `probinet` for this, which also contains algorithms to run community detection and other inference tasks

In [ ]:
undirected = True
force_dense = True
binary = True
data = build_adjacency_from_file(
    f"{outdir}{filename}",
    ego=source,
    alter=target,
    sep=",",
    undirected=undirected,
    force_dense=force_dense,
    binary=binary,
    header=0,
)
# Print the names of the coordinates in the namedtuple gdata
print(data._fields)

Check that there are no missing nodes from the original dataframe

In [ ]:
missing_nodes = set(nodes).difference(set(data.graph_list[0].nodes()))
assert len(missing_nodes) == 0, f"{df[(df[source].isin(missing_nodes)) | (df[target].isin(missing_nodes))]}"

In [ ]:
Y = data.adjacency_tensor
Y.shape, len(nodes)

In [ ]:
plt.figure(figsize=(18,6))

nmax = 500
node_order = np.argsort(-Y[0].sum(axis=1))
viz.plot_matrix(Y,node_order=node_order[:nmax],title=f"Y")

plt.tight_layout()

Check for isolated nodes

In [ ]:
degree = np.count_nonzero(Y[0],axis=1) # number of neighbors
non_isolates = np.count_nonzero(degree)
print(f"There are {non_isolates} non-isolated nodes over N = {len(data.nodes)}")

Check basic network statistics

In [ ]:
G = data.graph_list
print_graph_stats(G)

## 2. Run community detection

We use the following algorithms:
- [`Louvain`](https://doi.org/10.1088/1742-5468/2008/10/P10008): deterministic algorithm, based on modularity maximization
   - Blondel V.D. et al. _Fast unfolding of communities in large networks_. J. Stat. Mech 10008, 1-12, 2008
- [`MultiTensor`](https://doi.org/10.1103/PhysRevE.95.042317) (MT): probabilistic algorithm, based on tensor factorization, MLE
   - De Bacco C., Power E.A., Larremore D.B. and Moore C. _Community detection, link prediction, and layer interdependence in multilayer networks_. Physical Review E, 95(4): 042317, 2017
- [`NPDC`](https://doi.org/10.1103/PhysRevE.95.012317 ): probabilistic algorithm, based on a non-parametric model, degree-corrected, Bayesian
    - Peixoto TP. _Nonparametric Bayesian inference of the microcanonical stochastic block model_. Physical Review E 95(1):012317, 2017 
- [`MTCOV`](https://doi.org/10.1038/s41598-020-72626-y): probabilistic algorithm, based on tensor factorization. Similar to MT, but takes in input covariates. We give in input as covariate the result of another algorithm, to bias the result towards that local optimum
   - Contisciani M., Power E.A. and De Bacco C. _Community detection with node attributes in multilayer networks_. Scientific reports, 10(1):15736, 2020.

In [ ]:
u  = {} # here we store the membership vector for each algorithm

To warmup, we use the Louvain algorithm, included in `networkx`

Setup variables for plotting

In [ ]:
ms = 100
# node_size = [np.log(graph.degree[i]) * ms + 100 for i in data.nodes]
# position = nx.spring_layout(data.graph_list[0], iterations=100, seed = seed)

node_size = [graph.degree[i] * ms + 20 for i in data.nodes]
position = tl.get_custom_node_positions(data.graph_list[0])

Visualize the network

In [ ]:
plot_labels = False
filename0 = f'WTO_network_{plot_labels}'
node_labels = {}
for n,d in list(data.graph_list[0].degree()):
    if d > 2: node_labels[n] = n
        
plt.figure(figsize=(12,10))

nx.draw_networkx_nodes(data.graph_list[0],position, node_size=node_size, node_color=viz.default_colors_dict['blue'], edgecolors=viz.default_colors_dict['dark_grey'])
if plot_labels == True:
    nx.draw_networkx_labels(data.graph_list[0],position, font_size=14, alpha=0.8, labels=node_labels)
nx.draw_networkx_edges(data.graph_list[0],pos=position,width=0.1)
# plt.title(p)
plt.axis('off')

plt.tight_layout()

filename = tl.get_filename(filename0,lecture_id=lecture_id)
outdir = "../figures/"
tl.savefig(plt,outfile = filename,outdir = outdir)

# plt.show()

### 2.1 Louvain

In [ ]:
algo = 'louvain'
def from_louvain_to_u(louvain: list) -> np.ndarray:
    '''
    Builds one-hot encoded vector of dimension k=# groups
    '''
    N = sum([len(s) for s in louvain])
    K = len(louvain)
    u = np.zeros((N,K))
    for k, partition in enumerate(louvain):
        p = np.array(list(partition))
        u[p,k] = 1
    assert np.all(u.sum(axis=1)==1)
    return u

G = nx.from_numpy_array(Y[0],edge_attr=weight)
G.number_of_nodes(), G.number_of_edges()

seed = 10
resolution = 1.5 # the higher, the more and smaller the communities
louvain = nx.community.louvain_communities(G, seed=seed,weight=weight,resolution=resolution)

u[algo] = from_louvain_to_u(louvain)
print(u[algo].shape)

In [ ]:
communities = {"u": np.argmax(u[algo], axis=1)}
_ = plot_hard_membership(data.graph_list[0], communities, position, node_size, colors, viz.edge_color)

### 2.2 MultiTensor

Import results (not run here). 
If you want to run yourself, [here](https://github.com/MPI-IS/multitensor) is the code

In [ ]:
K = 8

In [ ]:
algo = 'mt'
infile = f'../../../data/input/wto_theta_mt_K8.npz'
theta = np.load(infile)
u[algo] = theta['u']

In [ ]:
communities = {"u": np.argmax(u[algo], axis=1)}
_ = plot_hard_membership(data.graph_list[0], communities, position, node_size, colors, viz.edge_color)

In [ ]:
communities = {"u": u[algo]}
_ = plot_soft_membership(data.graph_list[0], communities, position, [0.1 * s for s in node_size], colors, viz.edge_color)

### 2.3 Non-parametric DC
For this, we use the package `pysbm`
- Clone the github repository at https://github.com/funket/pysbm and save it in a local folder

In [ ]:
K = 8
N_real = 20

In [ ]:
degree_corrected_objective_function = pysbm.DegreeCorrectedUnnormalizedLogLikelyhood(is_directed=False)

In [ ]:
algo = 'npdc'
best_objective = -1000000
best_partition_standard_NPDC = None
for r in range(N_real):
    degree_corrected_partition = pysbm.NxPartition(graph=data.graph_list[0],number_of_blocks=K)
    degree_corrected_inference = pysbm.PeixotoInference(data.graph_list[0], degree_corrected_objective_function, degree_corrected_partition)
    degree_corrected_inference.infer_stochastic_block_model()
    L=degree_corrected_objective_function.calculate(degree_corrected_partition)
    if L>best_objective:
        best_objective=L
        best_partition_standard_NPDC=degree_corrected_partition
    # print(r,L,best_objective)

In [ ]:
def from_pysbm_partition(partition, nodes) -> np.ndarray:
    '''
    Builds one-hot encoded vector of dimension k=# groups
    '''
    communities = np.array([partition.get_block_of_node(node) for node in nodes])
    N = len(communities)
    K = len(np.unique(communities))
    u = np.zeros((N,K))
    for i,k in enumerate(communities):
        u[i,k] = 1
    assert np.all(u.sum(axis=1)==1)
    return u

u[algo] = from_pysbm_partition(degree_corrected_partition, data.nodes)

In [ ]:
communities = {"u": np.argmax(u[algo], axis=1)}
_ = plot_hard_membership(data.graph_list[0], communities, position, node_size, colors, viz.edge_color)

### 2.4 MTCOV

We use a model contained in [`probinet`](https://mpi-is.github.io/probinet/index.html). For this, we need to setup the configuration.

In [ ]:
config_dict = {
    "assortative": True,
    "end_file": "_mtcov",
    "out_folder": '../../../data/outdir/wto/',
    "out_inference": True,
    "undirected": True,
    "rseed": 10
}
num_realizations = 20
plot_loglik = True

In [ ]:
model = MTCOV(num_realizations=num_realizations, plot_loglik=plot_loglik)

X = np.copy(u['mt']) # we can choose what dummy covariate to give in input. Here we use the result of another algorithm, pick the one you like most
# X = np.zeros((len(data.nodes), 4)) # uncomment this if you want to give dummy data
data = data._replace(design_matrix=X)

In [ ]:
K = 8
params = model.fit(data, K=K, gamma=0.5, rng=np.random.default_rng(config_dict["rseed"]), **config_dict)


In [ ]:
algo = 'mtcov'
u[algo] = params[0]

In [ ]:
communities = {"u": np.argmax(u[algo], axis=1)}
_ = plot_hard_membership(data.graph_list[0], communities, position, node_size, colors, viz.edge_color)

## 3. Analyze results


In [ ]:
nodeLabel2Id = {k:i for i,k in enumerate(data.nodes)}
nodeId2Label = {i:k for i,k in enumerate(data.nodes)}

### 3.1 Communities

In [ ]:
node_labels = {}
for n,d in list(data.graph_list[0].degree()):
    if d > 2: node_labels[n] = n
        
plt.figure(figsize=(16,10))
L = len(u.keys())
n_cols = 2
n_rows = int(np.ceil(L / n_cols))

for i, p in enumerate(u.keys()):
    plt.subplot(n_rows,n_cols,i+1)
    nx.draw_networkx_nodes(data.graph_list[0],position, node_size=node_size, node_color=get_node_colors(colors, u[p]))
    nx.draw_networkx_labels(data.graph_list[0],position, font_size=8, alpha=0.8, labels=node_labels)
    nx.draw_networkx_edges(data.graph_list[0],pos=position,width=0.1)
    plt.title(p)
    plt.axis('off')


plt.show()

### 3.2 Adjacency matrices
Sorted by argmax of each node

In [ ]:
f, axarr = plt.subplots(1, len(u.keys()),figsize=(18,6))

for i,algo in enumerate(u.keys()):
    node_order = tl.extract_node_order(u[algo])
    viz.plot_matrix(Y,node_order=node_order,ax=axarr[i],title=f"{algo}",vmax = 1e-3,vmin=0)

plt.tight_layout()

### 3.3 Focus on a specific partition and zoom in

In [ ]:
nodeLabel2size = {i:np.log(data.graph_list[0].degree[i]) * ms +200 for i in list(data.graph_list[0].nodes())}

Play with the algorithm and reflect on how the different partitions compare.
- What do they capture?
- Are there patterns revealed by a praticular algorithm distinct from others?

In [ ]:
algo = 'mtcov'
communities = np.argmax(u[algo],axis=1)

In [ ]:
from probinet.visualization.plot import extract_bridge_properties

In [ ]:
plt.figure(figsize=(14,10))
K = u[algo].shape[-1]
n_cols = 4 
n_rows = int(np.ceil(K / n_cols))
for i, k in enumerate(np.arange(u[algo].shape[-1])):
    community = np.where(communities==k)[0]
    H = data.graph_list[0].subgraph([nodeId2Label[n] for n in community])
    c = colors[i]
    p = nx.spring_layout(H, iterations=100,k=0.1)
    ns = [nodeLabel2size[n] for n in H.nodes()]
    plt.subplot(n_rows,n_cols,i+1)
    nx.draw_networkx_edges(H,pos=p, width=0.1)
    nx.draw_networkx_labels(H,pos=p, font_size=8, alpha=0.8)
    if algo in ['mt','mtcov']:
        ax = plt.gca()
        for j, n in enumerate(H.nodes()):
            wedge_sizes, wedge_colors = extract_bridge_properties(j, colors, u[algo][communities==k])
            if len(wedge_sizes) > 0:
                _ = plt.pie(
                    wedge_sizes,
                    center=p[n],
                    colors=wedge_colors,
                    radius=(ns[j]) * 0.0003
                )
                ax.axis("equal")
    else:
        nx.draw_networkx_nodes(H,pos=p, node_size=ns, node_color=c)
        
    
    plt.title(k)
    plt.axis('off')
plt.tight_layout()


### 3.4 Alternative way to visualize communities

In [ ]:
L = len(u.keys())
ref_algo = 'mt'
node_order = tl.extract_node_order(u[ref_algo])
y_labels = [nodeId2Label[i] for i in node_order]
y_ticks = np.arange(len(node_order))

f, axarr = plt.subplots(1, L,figsize=(12,10), sharey=True)

for i, a in enumerate(u.keys()):
    axarr[i].imshow(u[a][node_order],aspect='auto',vmax=1, cmap='Blues' )
    axarr[i].set_title(f"{a}")
    axarr[i].set_xlabel('Community')

axarr[0].set_yticklabels(y_labels,fontsize=10)
axarr[0].set_yticks(y_ticks)
# plt.show()
plt.tight_layout()

In [ ]:
algo = 'mt'
plot_labels = True
filename0 = f'WTO_network_communities_{algo}_{plot_labels}'

ms = 100
ns = [np.log(graph.degree[i]) * ms + 150 for i in data.nodes]

node_labels = {}
for n,d in list(data.graph_list[0].degree()):
    if d > 1: node_labels[n] = n
        
plt.figure(figsize=(12,10))

# nx.draw_networkx_nodes(data.graph_list[0],position, node_size=node_size, node_color=default_colors['blue'], edgecolors=default_colors['dark_grey'])

nx.draw_networkx_edges(data.graph_list[0],pos=position,width=0.1)
if algo in ['mt','mtcov']:
    ax = plt.gca()
    for j, n in enumerate(data.graph_list[0].nodes()):
        wedge_sizes, wedge_colors = extract_bridge_properties(j, colors, normalize_nonzero_membership(u[algo]))
        if len(wedge_sizes) > 0:
            _ = plt.pie(
                wedge_sizes,
                center=position[n],
                colors=wedge_colors,
                radius=(ns[j]) * 0.0001, 
                wedgeprops=dict(edgecolor=viz.default_colors_dict['dark_grey'])
            )
            ax.axis("equal")
        else:
            print(j,n,u[algo][j])
else:
    nx.draw_networkx_nodes(data.graph_list[0],position, node_size=node_size, node_color=tl.get_node_colors(colors, u[p]), edgecolors=viz.default_colors_dict['dark_grey'])
if plot_labels == True:
    nx.draw_networkx_labels(data.graph_list[0],position, font_size=14, alpha=0.8, labels=node_labels)
            
# plt.title(p)
plt.axis('off')

plt.tight_layout()

filename = tl.get_filename(filename0,lecture_id=lecture_id)
outdir = "../figures/"
tl.savefig(plt,outfile = filename,outdir = outdir)

# plt.show()

# Appendix

## Toy model example

In [ ]:
from math import comb
import seaborn as sns

def calculate_loglikelihood_sbm(N: np.ndarray, M: np.ndarray)->dict:

    n_groups = len(N)
    N_rs = np.outer(N,N)
    for i in range(n_groups):
        N_rs[i,i] = comb(N[i],2)
    C = M / N_rs
    assert np.all(C<=1)
    assert np.all(C>=0)
    assert np.all(N_rs>=M)
    p = 1
    for r in np.arange(n_groups):
        p *= (np.power(C[r,r],M[r,r])) * (np.power(1-C[r,r],N_rs[r,r] - M[r,r]))
        for s in np.arange(r,n_groups):
            if s != r:
                p *= (np.power(C[r,s],M[r,s])) * (np.power(1-C[r,s],N_rs[r,s] - M[r,s]))
    return {'N_rs':N_rs, 'p':p,'C':C, 'logp':np.log(p)}

In [ ]:
# Left example
N = np.array([4,2])
K = N.shape[0]
M = 2 * np.ones([K,K])
np.fill_diagonal(M, [4,1])

# Right example
# N = np.array([3,3])
# K = N.shape[0]
# M = 1 * np.ones([K,K])
# np.fill_diagonal(M, [3,3])

print(M)
res = calculate_loglikelihood_sbm(N,M)
res

In [ ]:
plt.figure(figsize=(4,4))
sns.heatmap(res['C'],cmap='Blues')

In [ ]:
np.exp(-8.318),np.exp(-3.139)